In [2]:
using MCMRSimulator
using CairoMakie
using Statistics
using DelimitedFiles
using FileIO
using JLD2
using Printf
include("just_random_positions.jl")

just_random_position (generic function with 1 method)

In [8]:
using Random

function repelling_random_position(radius, pos_in, repeatsize; maxiter=1000, repulsion_strength=0.01)
    pos = deepcopy(pos_in)
    size = repeatsize
    # Initialize circles list
    
    function does_repulsed_have_a_collision(circle, index)
        if circle[1] > size/2 || circle[1] < -size/2
            return true
        elseif circle[2] > size/2 || circle[2] < -size/2
            return true
        end
        for i in 1:length(pos)
            if i != index
                other_circle = pos[i]
                a = 2*radius
                x = mod((circle[1] - other_circle[1] + size/2), size) - size/2
                y = mod((circle[2] - other_circle[2] + size/2), size) - size/2

                if a >= sqrt(x^2 + y^2)
                    return true
                end
            end
        end

        return false
    end
    

        # Update circle positions based on repulsion
    for a in 1:maxiter
        success_count=0
        for i in 1:length(pos)
            start_pos = pos[i]
            proposed_pos = pos[i]
            for j in 1:length(pos)
                if i != j
                    # ((abs(circle[1][1] - other_circle[1][1]) + size/2) % size) - size/2
                    dx = mod((pos[j][1] - pos[i][1] + size/2), size) - size/2
                    dy =  mod((pos[j][2] - pos[i][2] + size/2), size) - size/2
                    dist = sqrt(dx^2 + dy^2)
                    # force = (radius^2*radius^2)*repulsion_strength / (dist * mean_radius^2)
                    force = (radius^2)*repulsion_strength / (dist)
                    proposed_pos = (proposed_pos[1] - force * dx/dist, proposed_pos[2] - force * dy/dist)
                    # println(dx, dy)
                end
            end
            # Abandon changes if there's overlap
            if does_repulsed_have_a_collision(proposed_pos, i)
                pos[i] = start_pos
            else
                pos[i] = proposed_pos
                success_count += 1
            end
        end
        if success_count == 0
            println(a)
            break
        end
        # println(a)
    end
    # rng_pos = []
    # for i in 1:length(circles)
    #     push!(rng_pos, Float64.(circles[i][1]))
    # end
    # bounding_box = MCMRSimulator.BoundingBox([-size/2, -size/2, -500], [size/2, size/2, 500])
        
    return pos
end

repelling_random_position (generic function with 1 method)

In [10]:
T2s = 10:10:150
t_dwell = 1
TE = 20
T2_bound=1e-3
svratio = 2/2

for r in 0.3:0.1:5.5
    rho = 0.65
    rep = r*70
    res = MCMRSimulator.random_positions_radii([rep, rep], rho, 2, mean=r, variance=0)
    repeled = repelling_random_position(r, res[1], rep, maxiter=1000, repulsion_strength=3e-3)
    for T2 = T2s # ms
        surf_dens = 1/svratio*t_dwell/T2
        rng_geom = Cylinders(position=repeled, radius=r, dwell_time=t_dwell, density=surf_dens, repeats=[rep, rep], R2_surface=1/T2_bound)
        MCMRSimulator.write_geometry("./same_den_cylinders/cylinders_MT_"* string(T2) *"_sus_0_perm_0.000_rmean_"*@sprintf("%.2f",r)*"_density_0.65.json", rng_geom)
    end
    # for perm in 0:0.001:0.02
    #     rng_geom = Cylinders(position=repeled, radius=r, permeability=perm, repeats=[rep, rep])
    #     MCMRSimulator.write_geometry("./semifinal_cylinders/permeability/cylinders_MT_0_sus_0_perm_"*@sprintf("%.3f",perm)*"_rmean_"*@sprintf("%.2f",r)*"_density_0.65.json", rng_geom)
    # end
end

There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values for Field(position)
There are 1014 Cylinder based on values 